In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from sklearn.cluster import KMeans

In [ ]:
df = pd.read_csv("../scripts/eda_2018.csv")

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
threshold = 3
col = 'Departure Delay'

z_scores = np.abs((df[col] - df[col].mean()) / df[col].std())
df = df.loc[z_scores < threshold]
df.shape

In [ ]:
print(df[col].max())
print(df[col].mean())
print(df[col].median())
print(df[col].min())

In [ ]:
df['Departure Delay'] = df['Departure Delay'] - df['NAS Delay'] - df['Security Delay']

In [ ]:
FEATURES = [
    'Date', 'Month',
    'Origin Weather Code', 'Origin Temperature Max',
    'Origin Temperature Min', 'Origin Temperature Mean',
    'Origin Apparent Temperature Max', 'Origin Apparent Temperature Min',
    'Origin Apparent Temperature Mean', 'Origin Sunrise', 'Origin Sunset',
    'Origin Shortwave Radiation', 'Origin Precipitation', 'Origin Rain',
    'Origin Snowfall', 'Origin Precipitation Hours', 'Origin Windspeed',
    'Origin Windgusts', 'Origin Wind Direction',
    'Origin Evapotranspiration',
    'Departure Delay',
]

# n = 300000
# weather_df = df[FEATURES]
# sampled_weather_df = weather_df.sample(n)
# sampled_weather_df.shape

sampled_weather_df = df[FEATURES]

## Initial Correlation Matrix

In [ ]:
corr = sampled_weather_df.corr()

fig = px.imshow(
    corr,
    x=corr.columns,
    y=corr.columns,
    color_continuous_scale='tealrose', 
    title='Weather features Correlation Heatmap',
    template="ggplot2",
)

fig.update_layout(
    autosize=True,
    yaxis_nticks=len(list(corr.columns)),
    xaxis_nticks=len(list(corr.columns)),
)

fig.show()

## Monthly Macro Trends

In [ ]:
MEDIAN_WEATHER_COLS = [
    'Origin Temperature Max', 'Origin Temperature Min', 'Origin Temperature Mean',
    'Origin Apparent Temperature Max', 'Origin Apparent Temperature Min', 'Origin Apparent Temperature Mean', 
    'Origin Shortwave Radiation', 'Origin Precipitation', 'Origin Rain', 'Origin Snowfall',
    'Origin Windspeed', 'Origin Windgusts', 'Origin Wind Direction', 'Origin Evapotranspiration',
    'Departure Delay'
]

MODE_WEATHER_COLS = [
    'Origin Weather Code', 'Origin Precipitation Hours'
]

monthly_median_df = sampled_weather_df.groupby('Month')[MEDIAN_WEATHER_COLS].median().reset_index()
monthly_mode_df = sampled_weather_df.groupby('Month')[MODE_WEATHER_COLS].agg(lambda x:x.value_counts().index[0]).reset_index()


In [ ]:
monthly_median_df['Origin Monthly Median Temperature Range'] = monthly_median_df['Origin Temperature Max'] - monthly_median_df['Origin Temperature Min']
monthly_median_df['Origin Monthly Median Apparent Temperature Range'] = monthly_median_df['Origin Apparent Temperature Max'] - monthly_median_df['Origin Apparent Temperature Min']


In [ ]:
RENAMED_MONTHLY_WEATHER_COLS = {
    'Origin Shortwave Radiation':'Origin Monthly Median Shortwave Radiation', 
    'Origin Precipitation': 'Origin Monthly Median Precipitation', 
    'Origin Rain': 'Origin Monthly Median Rain',
    'Origin Snowfall': 'Origin Monthly Median Snowfall', 
    'Origin Windspeed': 'Origin Monthly Median Windspeed', 
    'Origin Windgusts': 'Origin Monthly Median Windgusts', 
    'Origin Wind Direction': 'Origin Monthly Median Wind Direction',
    'Origin Evapotranspiration': 'Origin Monthly Median Evapotranspiration', 
    'Departure Delay': 'Monthly Median Departure Delay'
}

SELECTED_MONTHLY_WEATHER_COLS = [
    'Month',
    'Origin Monthly Median Temperature Range',
    'Origin Monthly Median Apparent Temperature Range',
    'Origin Monthly Median Shortwave Radiation',
    'Origin Monthly Median Precipitation',
    'Origin Monthly Median Rain',
    'Origin Monthly Median Snowfall',
    'Origin Monthly Median Windspeed',
    'Origin Monthly Median Windgusts',
    'Origin Monthly Median Wind Direction',
    'Origin Monthly Median Evapotranspiration',
    'Monthly Median Departure Delay'
]

monthly_median_df.rename(columns=RENAMED_MONTHLY_WEATHER_COLS, inplace = True)
monthly_df = monthly_median_df[SELECTED_MONTHLY_WEATHER_COLS]
monthly_df['Origin Monthly Mode Weather Code'] = monthly_mode_df['Origin Weather Code']
monthly_df['Origin Monthly Mode Precipitation Hours'] = monthly_mode_df['Origin Precipitation Hours']

In [ ]:
tmp = sampled_weather_df[['Departure Delay', 'Month']]
new_monthly_df = tmp.merge(monthly_df.set_index('Month'), on='Month', how='left')
print(new_weather_df.shape)
new_monthly_df.head()

In [ ]:
corr = new_monthly_df.corr()

fig = px.imshow(
    corr,
    x=corr.columns,
    y=corr.columns,
    color_continuous_scale='tealrose', 
    template="ggplot2",
    title='Monthly Macro Weather Features Correlation Matrix Heatmap',
    text_auto=True,
)

fig.update_layout(
    autosize=True,
    yaxis_nticks=len(list(corr.columns)),
    xaxis_nticks=len(list(corr.columns)),
)

fig.show()

## Binning Continuous Values using Equal Frequency Binning

In [ ]:
num_bins = 5

bin_edges = pd.qcut(sampled_weather_df['Origin Precipitation Hours'], q=num_bins, labels=False, duplicates='drop')

sampled_weather_df['Origin Precipitation Hours (Equal)'] = bin_edges
sampled_weather_df['Origin Precipitation Hours (Equal)'].value_counts()

In [ ]:
fig = px.scatter(
    sampled_weather_df,
    x='Origin Precipitation Hours', 
    y='Departure Delay', 
    color='Origin Precipitation Hours (Equal)',
    color_continuous_scale='tealrose', 
    template="ggplot2",
    opacity=0.01,
    title='Scatter Plot after binning'
)
fig.update_layout(
    autosize=True
)
fig.show()

In [ ]:
sampled_weather_df['Origin Daylight'] = pd.to_datetime(sampled_weather_df['Origin Sunset']) - pd.to_datetime(sampled_weather_df['Origin Sunrise'])
sampled_weather_df['Origin Daylight'] = sampled_weather_df['Origin Daylight'].apply(lambda x:x.total_seconds())

num_bins = 4

bin_edges = pd.qcut(sampled_weather_df['Origin Daylight'], q=num_bins, labels=False, duplicates='drop')

sampled_weather_df['Origin Daylight (Equal)'] = bin_edges
sampled_weather_df['Origin Daylight (Equal)'].value_counts()


In [ ]:
fig = px.scatter(
    sampled_weather_df,
    x='Origin Daylight', 
    y='Departure Delay',
    color='Origin Daylight (Equal)', 
    color_continuous_scale='tealrose', 
    template="ggplot2"
)
fig.update_layout(title='Scatter Plot after binning')
fig.show()

In [ ]:
fig = px.scatter(
    sampled_weather_df,
    x='Origin Weather Code', 
    y='Departure Delay',
    color='Origin Weather Code', 
    color_continuous_scale='tealrose', 
    template="ggplot2",
    title='Scatter Plot after binning'
)
fig.update_layout(
    autosize=True
)
fig.show()

In [ ]:
BINNED_COLS = [
    'Origin Precipitation Hours (Equal)',
    'Origin Daylight (Equal)',
    'Origin Weather Code',
    'Departure Delay' 
]

new_binned_df = sampled_weather_df[BINNED_COLS]
corr = new_binned_df.corr()

fig = px.imshow(
    corr,
    x=corr.columns,
    y=corr.columns,
    color_continuous_scale='tealrose', 
    template="ggplot2",
    title='Binned Weather Features Correlation Matrix Heatmap',
    text_auto=True,
)

fig.update_layout(
    autosize=True,
    yaxis_nticks=len(list(corr.columns)),
    xaxis_nticks=len(list(corr.columns)),
)

fig.show()

## Weekly Macro Trends

In [ ]:
MEDIAN_WEATHER_COLS = [
    'Date',
    'Origin Temperature Max', 'Origin Temperature Min', 'Origin Temperature Mean',
    'Origin Apparent Temperature Max', 'Origin Apparent Temperature Min', 'Origin Apparent Temperature Mean', 
    'Origin Shortwave Radiation', 'Origin Precipitation', 'Origin Rain', 'Origin Snowfall',
    'Origin Windspeed', 'Origin Windgusts', 'Origin Wind Direction', 'Origin Evapotranspiration',
    'Departure Delay'
]

MODE_WEATHER_COLS = [
    'Date',
    'Origin Weather Code', 'Origin Precipitation Hours'
]
median_weather_df = sampled_weather_df[MEDIAN_WEATHER_COLS]
mode_weather_df = sampled_weather_df[MODE_WEATHER_COLS]
mode_weather_df

In [ ]:
median_weather_df['Date'] = pd.to_datetime(median_weather_df['Date'])
mode_weather_df['Date'] = pd.to_datetime(mode_weather_df['Date'])

median_weather_df.set_index('Date', inplace=True)
mode_weather_df.set_index('Date', inplace=True)

weekly_medians = median_weather_df.resample('W').median()
weekly_modes = mode_weather_df.resample('W').median()

print(weekly_medians.shape)
weekly_medians.columns

In [ ]:
print(weekly_modes.shape)
weekly_modes.head()

In [ ]:
RENAMED_WEEKLY_WEATHER_COLS = {
    'Origin Shortwave Radiation':'Origin Weekly Median Shortwave Radiation', 
    'Origin Precipitation': 'Origin Weekly Median Precipitation', 
    'Origin Rain': 'Origin Weekly Median Rain',
    'Origin Snowfall': 'Origin Weekly Median Snowfall', 
    'Origin Windspeed': 'Origin Weekly Median Windspeed', 
    'Origin Windgusts': 'Origin Weekly Median Windgusts', 
    'Origin Wind Direction': 'Origin Weekly Median Wind Direction',
    'Origin Evapotranspiration': 'Origin Weekly Median Evapotranspiration', 
    'Departure Delay': 'Weekly Median Departure Delay'
}

SELECTED_COLS = [
    'Date',
    'Origin Weekly Median Shortwave Radiation', 
    'Origin Weekly Median Precipitation', 
    'Origin Weekly Median Rain',
    'Origin Weekly Median Snowfall', 
    'Origin Weekly Median Windspeed', 
    'Origin Weekly Median Windgusts', 
    'Origin Weekly Median Wind Direction',
    'Origin Weekly Median Evapotranspiration', 
    'Weekly Median Departure Delay',
    'Origin Weekly Mode Weather Code',
    'Origin Weekly Mode Precipitation Hours',
    'Origin Weekly Median Temperature Range',
    'Origin Weekly Median Apparent Temperature Range',
]

weekly_df = weekly_medians.rename(columns=RENAMED_WEEKLY_WEATHER_COLS)
weekly_df['Origin Weekly Mode Weather Code'] = weekly_modes['Origin Weather Code']
weekly_df['Origin Weekly Mode Precipitation Hours'] = weekly_modes['Origin Precipitation Hours']

weekly_df['Origin Weekly Median Temperature Range'] = weekly_medians['Origin Temperature Max'] - weekly_medians['Origin Temperature Min']
weekly_df['Origin Weekly Median Apparent Temperature Range'] = weekly_medians['Origin Apparent Temperature Max'] - weekly_medians['Origin Apparent Temperature Min']
weekly_df.reset_index(inplace=True)
weekly_df = weekly_df[SELECTED_COLS]
weekly_df.head()

In [ ]:
count = 0
daily_df = pd.DataFrame([])
for week in weekly_df['Date']:
    start_date = week + pd.DateOffset(days=-6)
    end_date = week
    
    for date in pd.date_range(start_date, end_date, freq='D'):
        row = weekly_df.loc[count:count]
        row = row.assign(Day=str(date))
        daily_df = pd.concat([daily_df, row])
    count += 1

daily_df.drop(['Date'], axis=1, inplace=True)
daily_df['Day'] = pd.to_datetime(daily_df['Day'])
daily_df.rename(columns={'Day':'Date'}, inplace=True)
daily_df.head()

In [ ]:
tmp = sampled_weather_df[['Date', 'Departure Delay']]
tmp['Date'] = pd.to_datetime(tmp['Date'])

merged_weekly_df = tmp.merge(daily_df, on='Date', how='left')
merged_weekly_df.head()

In [ ]:
corr = merged_weekly_df.corr()

fig = px.imshow(
    corr,
    x=corr.columns,
    y=corr.columns,
    color_continuous_scale='tealrose', 
    template="ggplot2",
    title="Weekly Correlation Heatmap"
)

fig.update_layout(
    autosize=True,
    yaxis_nticks=len(list(corr.columns)),
    xaxis_nticks=len(list(corr.columns)),
)

fig.show()

In [ ]:
SELECTED_WEEKLY_COLS = [
    'Origin Weekly Median Temperature Range',
    'Origin Weekly Median Precipitation',
    'Origin Weekly Median Windspeed',
    'Origin Weekly Mode Weather Code',
    'Origin Weekly Mode Precipitation Hours',
    'Weekly Median Departure Delay',
    'Departure Delay'
]

selected_weekly_df = merged_weekly_df[SELECTED_WEEKLY_COLS]

corr = selected_weekly_df.corr()

fig = px.imshow(
    corr,
    x=corr.columns,
    y=corr.columns,
    color_continuous_scale='tealrose', 
    template="ggplot2",
    title="Weekly Correlation Heatmap"
)

fig.update_layout(
    autosize=True,
    yaxis_nticks=len(list(corr.columns)),
    xaxis_nticks=len(list(corr.columns)),
)

fig.show()

In [ ]:
selected_weekly_df['Weekly Median Departure Delay'].value_counts()

## Standardising Weather Features